Meta, Apple and Tesla stock news are extracted using beautifulsoup

In [1]:
import pandas as pd
import sys
import requests
from bs4 import BeautifulSoup

In [29]:
limit = 3 # setting to 3 articles limit per page

def getDate(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    dates = soup.find_all(class_ = "date") 
    dateArray=[]
    count = 1
    for date in dates :
        dateArray.append(date.get_text().split()[1:])
        if count >= limit :
            break
        count += 1    
    return dateArray

# tweak start and end for faster compile
def getStartEnd(url,startnum,endnum):
    try :
        check = True
        start = getDate("{}{}".format(url,startnum))
        while check :
            if (start[2][0] == 'Aug' and start[2][2] == '2022') :
                check = False
                break
            startnum += 1
            start = getDate("{}{}".format(url,startnum))
            print(start)
        print(startnum)
        # coronavirus special - oldest date to mar 2020
        if url == 'https://www.investing.com/news/coronavirus/' :
            endnum = 650 # last updated 17 October 2022
        else :
            check = True
            end = getDate("{}{}".format(url,endnum))
            while check :
                if end[2][0] == 'Jan' and end[2][2] == '2018' :
                    check = False
                    break
                endnum += 1
                end = getDate("{}{}".format(url,endnum))    
                print(end)
            print(endnum)
            return (startnum,endnum)
    except : 
        print("error")

def getArticles(url,startnum,endnum):
    df = pd.DataFrame(columns=['Date','Title','Text'])
    start,end = getStartEnd(url,startnum,endnum)
    while start <= end :
        page = requests.get("{}{}".format(url,start))
        soup = BeautifulSoup(page.content, 'html.parser')
        dates = soup.find_all(class_="date")
        titles = soup.find_all(class_ = "title") 
        texts = soup.find_all("p")
        nTexts = []
        textCount = 0
        for t in texts :
            if t.text[:2] == "By":
                nTexts.append(t.text)
                textCount += 1
            if textCount == limit :
                break
        for i in range(textCount) :
            df_new = pd.DataFrame({"Date" : dates[i].text[3:], "Title" : titles[i].text, "Text" : nTexts[i]},index=[1])
            df = pd.concat([df,df_new],ignore_index=True)
        print(start, end, "[Page {}/{}]".format(start, end))
        start += 1
    return df

In [41]:
df_world = getArticles("https://www.investing.com/news/world-news/",98,4313)

98
[['Feb', '04,', '2018'], ['Feb', '04,', '2018'], ['Feb', '04,', '2018']]
[['Feb', '03,', '2018'], ['Feb', '03,', '2018'], ['Feb', '03,', '2018']]
[['Feb', '03,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018']]
[['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018']]
[['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018']]
[['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018']]
[['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018']]
[['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018']]
[['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018']]
[['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018']]
[['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018']]
[['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018']]
[['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018']]
4313
98 4313 [Page 98/4313]
99 4313 [Page 99/431

KeyboardInterrupt: 

In [48]:
df_politics = getArticles("https://www.investing.com/news/politics/",9,780) 

9
[['Feb', '16,', '2018'], ['Feb', '16,', '2018'], ['Feb', '16,', '2018']]
[['Feb', '15,', '2018'], ['Feb', '15,', '2018'], ['Feb', '15,', '2018']]
[['Feb', '14,', '2018'], ['Feb', '14,', '2018'], ['Feb', '14,', '2018']]
[['Feb', '13,', '2018'], ['Feb', '12,', '2018'], ['Feb', '12,', '2018']]
[['Feb', '09,', '2018'], ['Feb', '09,', '2018'], ['Feb', '09,', '2018']]
[['Feb', '08,', '2018'], ['Feb', '08,', '2018'], ['Feb', '08,', '2018']]
[['Feb', '06,', '2018'], ['Feb', '06,', '2018'], ['Feb', '06,', '2018']]
[['Feb', '05,', '2018'], ['Feb', '05,', '2018'], ['Feb', '05,', '2018']]
[['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018']]
[['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018']]
780
9 780 [Page 9/780]
10 780 [Page 10/780]
11 780 [Page 11/780]


KeyboardInterrupt: 

In [61]:
df_coronavirus = getArticles("https://www.investing.com/news/coronavirus/",3,650) 

3
[['Mar', '31,', '2020'], ['Mar', '31,', '2020'], ['Mar', '31,', '2020']]
[]
error


TypeError: cannot unpack non-iterable NoneType object

In [6]:
df_world.to_csv('../Data/Global/world_news.csv',index=False)
df_politics.to_csv('../Data/Global/politics_news.csv',index=False)
df_coronavirus.to_csv('../Data/Global/coronavirus_news.csv',index=False)